In [3]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [ ]:

# Load the model
loaded_model = xgb.Booster()
loaded_model.load_model("xgb_popularity_model.json")

print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
import librosa
import numpy as np
import pandas as pd

# Path to your audio file
filename = '/Users/markol/Downloads/proba.mp3'

# Load audio
y, sr = librosa.load(filename, sr=None)


tempo = librosa.beat.tempo(y=y, sr=sr)[0]
energy = np.mean(librosa.feature.rms(y=y))
loudness = 20 * np.log10(energy)

# chroma
chroma = librosa.feature.chroma_cqt(y=y, sr=sr)

# key as an integer
key_strength = np.sum(chroma, axis=1)  # Sum chroma bins
key_index = np.argmax(key_strength)  # Index of the strongest key (integer representation)
key = key_index  # Save key as an integer

# mode (major/minor)
mode = 1 if key_strength[key_index] > np.mean(key_strength) else 0  # Major = 1, Minor = 0

# duration in [ms]
duration_ms = int(librosa.get_duration(y=y, sr=sr) * 1000)

# FEATURES
features = {
    'energy': [energy],
    'key': [key],  # Key as integer
    'loudness': [loudness],
    'mode': [mode],  # Major (1) or Minor (0)
    'tempo': [tempo],
    'duration_ms': [duration_ms]  # Duration in milliseconds
}

# Create DataFrame
df = pd.DataFrame(features)

# Save to CSV
output_path = 'inference_results.csv'
df.to_csv(output_path, index=False)

print(f"Features including duration_ms saved to: {output_path}")


/var/folders/48/f5vvvqhn21dg8qqqqyd_7fdm0000gn/T/ipykernel_18892/3986921889.py:12: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=y, sr=sr)[0]


Features including duration_ms saved to: inference_results.csv


In [8]:
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb

xgb_model = xgb.Booster()
xgb_model.load_model("xgb_popularity_model.json")  # Load the JSON model

scaler = joblib.load("scaler.joblib")  

new_song_features = df[ ['energy', 'key', 'loudness', 'mode', 'tempo', 'duration_ms']]  # Adjust columns if needed



In [10]:
new_song_scaled = scaler.transform(new_song_features)


/Users/markol/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
# predict
predictions = xgb_model.predict(new_song_scaled)
df['predicted_popularity'] = predictions
print(df)